# Plotting Tara Oceans sample locations

This notebook shows how to plot geospatial metadata attached to the samples gathered by the Tara Oceans project, data published in [Quantitative 3D-imaging for cell biology and ecology of environmental microbial eukaryotes](https://elifesciences.org/articles/26066).

Each plate in the [Tara Oceans dataset](https://idr.openmicroscopy.org/webclient/?show=screen-1201) contains multiple microscopy images taken from a water sample. Each plate has metadata stored as key-value pairs, including the coordinates of the water sample location.

### Dependencies
- [folium](https://github.com/python-visualization/folium)
- [pandas](https://pandas.pydata.org/)

In [3]:
import folium
import omero.clients
from idr import connection

### Connect to IDR server

In [10]:
conn = connection('idr.openmicroscopy.org')

Connected to IDR ...


### Load the data and the associates geospatial coordinates

In [11]:
screen = conn.getObject('Screen', 1201)
coords = []
for p in screen.listChildren():
    try:
        d = dict(p.getAnnotation().getValue())
    except AttributeError:
        continue
    if all ((f in d) for f in (
        'EVENT_LATITUDE_Start',
        'EVENT_LONGITUDE_Start',
        'EVENT_LATITUDE_End',
        'EVENT_LONGITUDE_End',
    )):
        co = (
            p.getId(),
            p.getName(),
            float(d['EVENT_LATITUDE_Start']),
            float(d['EVENT_LONGITUDE_Start']),
            float(d['EVENT_LATITUDE_End']),
            float(d['EVENT_LONGITUDE_End']),
        )
        coords.append(co)

### Disconnect when done loading data

In [12]:
conn.close()

### Create a folium feature for each sample

In [13]:
features = folium.FeatureGroup(name="Samples")
for coord in coords:
    line = [
        (coord[2], coord[3]),
        (coord[4], coord[5]),
    ]
    label = '<a href="https://idr.openmicroscopy.org/webclient/?show=plate-%d" target="_blank">Plate %s</a>' % coord[:2]
    sample = folium.PolyLine(line, popup=label, color='red', weight=10)
    features.add_child(sample)

### Create and display the map. 

Clicking on each point will popup the plate name and a link to the IDR data.

In [14]:
m = folium.Map(location=[0, 0], zoom_start=2,)
m.add_child(features)
m

### License
Copyright (C) 2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.